# Particionamento de Dados

Nesse notebook serão realizados testes de benchmark em algumas colunas com o objetivo de identificar se o particionamento Hive-Style via Spark tem o mesmo desempenho que o particionamento via Z-Order + Optimize do Delta.

### Queries

Aqui temos as queries que serão realizadas:



### Como sera o benchmark?

Benchmark será feito por meio de teste de tempo e teste de files skipped. 

### Máquina em que foi executado:

X

Quantas vezes um ônibus passa por um ponto de ônibus com lat e long x,y?
Quantos ônibus ativos e qual a velocidade média deles na cidade de Curitiba numa segunda?
Avg de velocidade e etc

1 - Qual são as linhas com maior velocidade média? (AVG)
2 - Dos ônibus que rodam o dia inteiro, quantos deles são ativos de manha? (COUNT) e qual a velocidade média deles?
3 - Quais linhas tem mais veiculos?
4 - Velocidade média por dia (filter and avg)

# TODO:

* Transformar os dados em um único schema
* Manter dois repos separados para teste
* 1 - Testar apenas o particionamento de data
* 2 - Testar o particionamento de hora
* 3 - Testar o particionamento de id_onibus
* 4 - Testar o particionamento de cidade
* 5 - Testar o z-ordering nas mesmas colunas acima
* 6 - Testar primeiro particionar e depois dar um z-ordering nos dados

## Preparação:

Definir quais podem ser as principais queries

## Resultados:

Tempo para cada um dos z-ordering, visualizando qual o menor tempo

## Finalização

Transferir tudo para o Databricks

In [94]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("particionamento").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/topicos_dados/lake/bronze"
SILVER_PATH = "/home/felipe/code/topicos_dados/lake/silver"
BRBUS_PATH = "/home/felipe/code/topicos_dados/lake/silver/silver_brbus/"

brbus = spark.read.format("delta").load(BRBUS_PATH)
brbus.show()

spark.stop()


+---------+---------+----------+--------------+--------+---------+-------------+-------------------+-------------------+
| latitude|longitude|    bus_id|          city|bus_code|bus_speed|bus_direction|         queried_at|         updated_at|
+---------+---------+----------+--------------+--------+---------+-------------+-------------------+-------------------+
|-22.94364|-43.25486|RJO_A50159|Rio de Janeiro|     220|      0.0|         NULL|2024-02-02 08:03:00|2024-02-02 08:01:48|
|-23.01045| -43.2975|RJO_C50118|Rio de Janeiro|     302|      8.0|         NULL|2024-02-02 08:03:00|2024-02-02 08:01:48|
|-22.93032|-43.23769|RJO_A50025|Rio de Janeiro|     220|      0.0|         NULL|2024-02-02 08:03:00|2024-02-02 08:01:48|
|-23.00146|-43.36449|RJO_C50047|Rio de Janeiro|   SP805|     16.0|         NULL|2024-02-02 08:03:00|2024-02-02 08:01:48|
|-22.92475|-43.25124|RJO_A50187|Rio de Janeiro|     608|     27.0|         NULL|2024-02-02 08:03:00|2024-02-02 08:01:49|
|-22.92565|-43.24485|RJO_A50127|

In [2]:
from pyspark.sql import *
from delta import *

builder = SparkSession.builder.appName("particionamento").config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/topicos_dados/lake/bronze"
SILVER_PATH = "/home/felipe/code/topicos_dados/lake/silver"
BRBUS_PATH = "/home/felipe/code/topicos_dados/lake/silver/silver_brbus/"

brbus = spark.read.format("delta").load(BRBUS_PATH)
brbus.createOrReplaceTempView("teste_brbus")


In [68]:
%%time

# 1 - Qual são as linhas com maior velocidade média? (AVG)
spark.sql("SELECT bus_code, AVG(bus_speed) AS media_speed FROM teste_brbus WHERE city = 'Rio de Janeiro' GROUP BY bus_code ORDER BY media_speed DESC LIMIT 5;").collect()

CPU times: user 1.75 ms, sys: 1.14 ms, total: 2.89 ms
Wall time: 760 ms


[Row(bus_code='SN388', media_speed=38.45345155161495),
 Row(bus_code='SN397', media_speed=36.0497900419916),
 Row(bus_code='SR388', media_speed=31.722706691262786),
 Row(bus_code='SN554', media_speed=31.6066852367688),
 Row(bus_code='SV335', media_speed=31.398428731762063)]

In [62]:
%%time 

spark.sql("SELECT bus_code, AVG(bus_speed) AS media_speed FROM teste_brbus WHERE city = 'Brasilia' AND queried_at BETWEEN '2024-02-02 08:03:00' AND '2024-02-02 22:03:00' GROUP BY bus_code ORDER BY media_speed DESC LIMIT 5")

CPU times: user 962 µs, sys: 655 µs, total: 1.62 ms
Wall time: 8.39 ms


DataFrame[bus_code: string, media_speed: double]

In [69]:
%%time
# 4 - Velocidade média por dia (filter and avg)

spark.sql("SELECT DATE(queried_at) AS data, AVG(bus_speed) AS media_speed FROM teste_brbus WHERE city = 'Rio de Janeiro' GROUP BY DATE(queried_at) ORDER BY data;").collect()


CPU times: user 1.99 ms, sys: 1.26 ms, total: 3.25 ms
Wall time: 813 ms


[Row(data=datetime.date(2024, 1, 31), media_speed=17.63520752282458),
 Row(data=datetime.date(2024, 2, 1), media_speed=12.616950252677453),
 Row(data=datetime.date(2024, 2, 2), media_speed=15.630655713834075)]

In [71]:
%%time

# 2 - Dos ônibus que rodam o dia inteiro, quantos deles são ativos de manha? (COUNT) e qual a velocidade média deles?

spark.sql("SELECT DATE(queried_at) AS data,        COUNT(DISTINCT bus_id) AS quantidade_onibus,       AVG(bus_speed) AS media_velocidade FROM teste_brbus WHERE city = 'Rio de Janeiro' AND HOUR(queried_at) >= 6  AND HOUR(queried_at) < 13 GROUP BY DATE(queried_at) ORDER BY data;")

CPU times: user 2.56 ms, sys: 1.57 ms, total: 4.14 ms
Wall time: 800 ms


[Row(data=datetime.date(2024, 2, 2), quantidade_onibus=3944, media_velocidade=16.645269760010176)]

In [85]:
%%time

# 3 - Quais linhas tem mais veiculos?
spark.sql("SELECT bus_code, COUNT(DISTINCT bus_id) AS quantidade_veiculos \
FROM teste_brbus \
WHERE city = 'Rio de Janeiro' \
  AND DATE(queried_at) = '2024-02-02' \
GROUP BY bus_code \
ORDER BY COUNT(DISTINCT bus_id) DESC \
LIMIT 5; \
")

CPU times: user 1.64 ms, sys: 0 ns, total: 1.64 ms
Wall time: 7.47 ms


DataFrame[bus_code: string, quantidade_veiculos: bigint]

In [93]:
%%time

# Filtrando os dados
df_filtered = brbus.filter((brbus['city'] == 'Rio de Janeiro') &
                        (F.hour(brbus['queried_at']) >= 6) &
                        (F.hour(brbus['queried_at']) < 13))

# Criando a coluna 'data' com a data de 'queried_at'
df_filtered = df_filtered.withColumn('data', F.to_date('queried_at'))

# Calculando a contagem de ônibus distintos e a média da velocidade
df_result = df_filtered.groupBy('data').agg(F.countDistinct('bus_id').alias('quantidade_onibus'),
                                             F.avg('bus_speed').alias('media_velocidade'))

# Ordenando os resultados pela data
df_result = df_result.orderBy('data')

# Exibindo o resultado
# df_result.show()

CPU times: user 1.56 ms, sys: 2.68 ms, total: 4.24 ms
Wall time: 16.8 ms


Os snippets acima são as queries, a partir daqui iremos realizar os testes de partição:

In [ ]:
df.repartition(F.col("country")).write.partitionBy("country").format(
    "delta"
).saveAsTable("country_people")
    